In [ ]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm


In [ ]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:35<00:00, 86.9MiB/s]


In [ ]:
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 1,541,570,560 parameters.


In [ ]:
!wget https://asr.iitm.ac.in/Gramvaani/NEW/GV_Eval_3h.tar.gz
!sudo tar -xvf /content/GV_Eval_3h.tar.gz

--2024-04-20 02:24:02--  https://asr.iitm.ac.in/Gramvaani/NEW/GV_Eval_3h.tar.gz
Resolving asr.iitm.ac.in (asr.iitm.ac.in)... 103.158.43.117
Connecting to asr.iitm.ac.in (asr.iitm.ac.in)|103.158.43.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62063203 (59M) [application/x-gzip]
Saving to: ‘GV_Eval_3h.tar.gz’

GV_Eval_3h.tar.gz   100%[===================>]  59.19M  11.1MB/s    in 7.0s    

2024-04-20 02:24:10 (8.50 MB/s) - ‘GV_Eval_3h.tar.gz’ saved [62063203/62063203]

GV_Eval_3h/
GV_Eval_3h/mp3.scp
GV_Eval_3h/Audio/
GV_Eval_3h/Audio/01-10385-01.mp3
GV_Eval_3h/Audio/01-07698-02.mp3
GV_Eval_3h/Audio/01-10039-03.mp3
GV_Eval_3h/Audio/01-06541-02.mp3
GV_Eval_3h/Audio/01-04113-03.mp3
GV_Eval_3h/Audio/01-06017-03.mp3
GV_Eval_3h/Audio/01-08119-03.mp3
GV_Eval_3h/Audio/01-02202-01.mp3
GV_Eval_3h/Audio/02-12441-02.mp3
GV_Eval_3h/Audio/02-19554-01.mp3
GV_Eval_3h/Audio/02-17411-01.mp3
GV_Eval_3h/Audio/01-02700-01.mp3
GV_Eval_3h/Audio/01-02028-02.mp3
GV_Eval_3h/Audio/

In [ ]:
result = model.transcribe("/content/GV_Eval_3h/Audio/01-00004-02.mp3", language='Hindi')

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
result["text"]

' वहीं राहुल कुमार बुकेश कुमार राजा कुमार अनुरख कुमार समी कुमार रहेश कुमार मधिस कुमार शाहित गर्जेनों कारपयता भी सामिल से जन्मवक दिलपांडे के मौधरा से बिहार मुखलवानी लिजर प्रेस्टूर्स'

In [ ]:
result1 = model.transcribe("/content/GV_Eval_3h/Audio/01-00071-02.mp3", )

In [ ]:
result1["text"]

In [ ]:
from os import path
#col = ['file', 'text']
import pandas as pd
import os.path as path

test_df = pd.read_csv('/content/GV_Eval_3h/text', header = None)
random_df = test_df.sample(n=10)
random_df[['file', 'text']] = random_df[0].str.split(' ', n=1, expand=True)
random_df.drop(0, axis = 1, inplace = True)
random_df['file'] = random_df['file'].astype("string")

random_df['file'] = random_df['file'].apply(lambda x: x + '.mp3')
random_df['file'] = random_df['file'].apply(lambda s: path.join('/content/GV_Eval_3h/Audio/', s))

In [ ]:
random_df

,file,text
880,/content/GV_Eval_3h/Audio/02-17067-01.mp3,के अवसर पर किया जा रहा है और सभी छे प्रखंडों म...
991,/content/GV_Eval_3h/Audio/02-22322-01.mp3,हो गयी है तापमान में बहुत अधिक गिरावट नहीं होन...
906,/content/GV_Eval_3h/Audio/02-18071-01.mp3,हम लोगों को वेतन नहीं दिया जा रहा हैं इधर जिस...
176,/content/GV_Eval_3h/Audio/01-03355-01.mp3,मतदाता केंद्र के बी एल ओ अथवा प्रखण्ड विकाश पद...
231,/content/GV_Eval_3h/Audio/01-04407-01.mp3,फिर कुछ दिन बंद कर देते हैं फिर वही नियम चालू...
323,/content/GV_Eval_3h/Audio/01-06049-02.mp3,लगी है कुछ दिनों पुर्व को मुंगेर की आवाज़ से शौ...
43,/content/GV_Eval_3h/Audio/01-00946-01.mp3,दिल की बातों उसे दे रही मात मात से कोई बनेगी न...
902,/content/GV_Eval_3h/Audio/02-17956-01.mp3,पैंतालीस लाख को पार कर जायेगा केंद्र ने बदली न...
755,/content/GV_Eval_3h/Audio/02-12881-01.mp3,हेलो सर मैं वृद्धा पेंशन के बारे में बहुत बार ...
290,/content/GV_Eval_3h/Audio/01-05357-03.mp3,किन्तु ज्यादा तर एटीएम में ब्रॉड नहीं होते


In [ ]:
def pred_rec(path):
  result = model.transcribe(path, language='Hindi', fp16=False)
  pred = result["text"]
  return pred
random_df['predict'] = random_df.iloc[:10, 0].apply(pred_rec)

In [ ]:
random_df.head(10)

,file,text,predict
880,/content/GV_Eval_3h/Audio/02-17067-01.mp3,के अवसर पर किया जा रहा है और सभी छे प्रखंडों म...,पर किया जा रहा है और सभी छे प्रखंडों में प्रख...
991,/content/GV_Eval_3h/Audio/02-22322-01.mp3,हो गयी है तापमान में बहुत अधिक गिरावट नहीं होन...,हो गई है तापमान में बहुत अधिक गिरावट नहीं होन...
906,/content/GV_Eval_3h/Audio/02-18071-01.mp3,हम लोगों को वेतन नहीं दिया जा रहा हैं इधर जिस...,से हम लोगों को वेदन नहीं दिया जा रहा है इधर ज...
176,/content/GV_Eval_3h/Audio/01-03355-01.mp3,मतदाता केंद्र के बी एल ओ अथवा प्रखण्ड विकाश पद...,मस्त दाता केंदर के विहेलो अथवा प्रखंड विखापत ...
231,/content/GV_Eval_3h/Audio/01-04407-01.mp3,फिर कुछ दिन बंद कर देते हैं फिर वही नियम चालू...,तो इसी तो वह खिर कुछ गिन बंद कर देता फिर वहीं...
323,/content/GV_Eval_3h/Audio/01-06049-02.mp3,लगी है कुछ दिनों पुर्व को मुंगेर की आवाज़ से शौ...,लगी है कुछ दिनों पूर्ण मुंगे की आवाब से सौचाल...
43,/content/GV_Eval_3h/Audio/01-00946-01.mp3,दिल की बातों उसे दे रही मात मात से कोई बनेगी न...,"जिल की बात उसे दे रही मात, मात से कोई बनेगी न..."
902,/content/GV_Eval_3h/Audio/02-17956-01.mp3,पैंतालीस लाख को पार कर जायेगा केंद्र ने बदली न...,पैतारिस लाग को पार कर जाएगा केंदरनी बदली निती...
755,/content/GV_Eval_3h/Audio/02-12881-01.mp3,हेलो सर मैं वृद्धा पेंशन के बारे में बहुत बार ...,अलो सार मैं द्वापिल सिल के बारे में बहुत बार ...
290,/content/GV_Eval_3h/Audio/01-05357-03.mp3,किन्तु ज्यादा तर एटीएम में ब्रॉड नहीं होते,किन्तु ज़ादातर एकीम में ग्राज नहीं होते


In [ ]:
random_df.iloc[:10, 1].to_csv('ref.txt', sep='\t', index=False, header = None)
random_df.iloc[:10, 2].to_csv('hyp.txt', sep='\t', index=False, header = None)

In [ ]:
!pip install asr-evaluation

In [ ]:
!wer /content/ref.txt /content/hyp.txt

Sentence count: 10
WER:    44.206% (       103 /        233)
WRR:    60.944% (       142 /        233)
SER:   100.000% (        10 /         10)


In [ ]:
!wget https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/vistaar/kathbath.zip


--2024-04-20 03:28:27--  https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/vistaar/kathbath.zip
Resolving objectstore.e2enetworks.net (objectstore.e2enetworks.net)... 101.53.136.18, 164.52.206.155, 101.53.152.33, ...
Connecting to objectstore.e2enetworks.net (objectstore.e2enetworks.net)|101.53.136.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9187772731 (8.6G) [application/zip]
Saving to: ‘kathbath.zip’

kathbath.zip        100%[===================>]   8.56G  12.5MB/s    in 12m 8s  

2024-04-20 03:40:37 (12.0 MB/s) - ‘kathbath.zip’ saved [9187772731/9187772731]



In [ ]:
!unzip -q /content/kathbath.zip

In [ ]:
kabath_df = pd.read_csv('/content/kathbath/hindi/test_known/bucket.csv')
kabath_df['file_path'] = kabath_df['file_path'].astype("string")
kabath_df['file_path'] = kabath_df['file_path'].str.replace('/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e_data/pilot4_karya/aug_dc_data_raw/dc_data_clean_wav', '/content/kathbath')


In [ ]:
kabath_rand = kabath_df.sample(n = 10)

In [ ]:
kabath_rand

,file_path,transcript,length
12,/content/kathbath/hindi/test_known/audio/384/8...,इस बार गंगा का अवतरण दिवस सभी पुण्य योगों के स...,154552
2877,/content/kathbath/hindi/test_known/audio/390/8...,ये जर्मनी से थे जो डेनमार्क में बस गए थे,102539
364,/content/kathbath/hindi/test_known/audio/888/8...,तब से ये लोग माओवादियों के खिलाफ पुलिस के साथ ...,57957
61,/content/kathbath/hindi/test_known/audio/277/8...,संपूर्ण समर्पण और त्याग का एक पर्व है छठ,68360
2559,/content/kathbath/hindi/test_known/audio/1052/...,उन के महापरिनिर्वाण पश्चात संघ का आकार में व्य...,138205
938,/content/kathbath/hindi/test_known/audio/277/8...,और पैरों की कमियों को एक लंबी स्कर्ट से छुपाया...,69103
2538,/content/kathbath/hindi/test_known/audio/705/8...,एक ही व्यक्ति कई रूपों में व्यवहार करता हुआ पा...,67988
1458,/content/kathbath/hindi/test_known/audio/888/8...,निर्भया के भाई हाल ही में पायलट बने थे,39753
1977,/content/kathbath/hindi/test_known/audio/929/8...,गुरुकुल की तर्ज पर स्कूल और कॉलेजों की शिक्षा ...,98453
776,/content/kathbath/hindi/test_known/audio/326/8...,मुफ्त घर और निर्माण मचान के आसपास काम करने के ...,89536


In [ ]:
kabath_rand['predict'] = kabath_rand.iloc[:10, 0].apply(pred_rec)

In [ ]:
kabath_rand

,file_path,transcript,length,predict
12,/content/kathbath/hindi/test_known/audio/384/8...,इस बार गंगा का अवतरण दिवस सभी पुण्य योगों के स...,154552,इस बार गंगा का अवतरण दिवस सभी पुर्ण्य योगों क...
2877,/content/kathbath/hindi/test_known/audio/390/8...,ये जर्मनी से थे जो डेनमार्क में बस गए थे,102539,एज जर्मनी से थे जो डेनमार्क में बस गए थे
364,/content/kathbath/hindi/test_known/audio/888/8...,तब से ये लोग माओवादियों के खिलाफ पुलिस के साथ ...,57957,तब से ये लोग मावादियों की खिलाफ पुलिस के साथ ...
61,/content/kathbath/hindi/test_known/audio/277/8...,संपूर्ण समर्पण और त्याग का एक पर्व है छठ,68360,संपुन समर्पन और त्या का एक फर्व है जट
2559,/content/kathbath/hindi/test_known/audio/1052/...,उन के महापरिनिर्वाण पश्चात संघ का आकार में व्य...,138205,उनके महापरि निर्वान पस्चात संग का अकार में व्...
938,/content/kathbath/hindi/test_known/audio/277/8...,और पैरों की कमियों को एक लंबी स्कर्ट से छुपाया...,69103,और पेरों की कमियों को एक लंबी स्कर्ट से छुपाय...
2538,/content/kathbath/hindi/test_known/audio/705/8...,एक ही व्यक्ति कई रूपों में व्यवहार करता हुआ पा...,67988,एक ही व्यक्ति कई रूपों विभार करता हुआ पाया जा...
1458,/content/kathbath/hindi/test_known/audio/888/8...,निर्भया के भाई हाल ही में पायलट बने थे,39753,निर्भ्या के भाई हाली में पारेट बने थे
1977,/content/kathbath/hindi/test_known/audio/929/8...,गुरुकुल की तर्ज पर स्कूल और कॉलेजों की शिक्षा ...,98453,गुरुकोल की तरच पर स्कूल और कॉलेजों की शिक्षा ...
776,/content/kathbath/hindi/test_known/audio/326/8...,मुफ्त घर और निर्माण मचान के आसपास काम करने के ...,89536,मुफ्त घर और निर्माण मचान के आसपास काम करने के...


In [ ]:
kabath_rand.iloc[:10, 1].to_csv('kabath_ref.txt', sep='\t', index=False, header = None)
kabath_rand.iloc[:10, 3].to_csv('kabath_hyp.txt', sep='\t', index=False, header = None)

In [ ]:
!wer /content/kabath_ref.txt /content/kabath_hyp.txt

Sentence count: 10
WER:    24.138% (        28 /        116)
WRR:    75.862% (        88 /        116)
SER:    90.000% (         9 /         10)


In [ ]:
random_df.drop('file', axis = 1).to_csv('test_wer.txt', sep = ',', index=False)
kabath_rand.drop(['file_path', 'length'], axis = 1).to_csv('kabath_wer.txt', sep = ',', index=False)